<a href="https://colab.research.google.com/github/Chansikan/do_not_split_small_sample/blob/main/1)Create_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Radiomics machine learning study with a small sample size: Single random training-test set split may lead to unreliable results: Creating datasets for analyses

In [2]:
# import necessary modules
import numpy as np
import pandas as pd
from tqdm import tqdm

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# run an external python code for DeLong method
# Upload a file named 'delong.py' onto the working directory
%run 'delong.py'

In [3]:
# Upload 4 files onto the working directory:
# 'GvM_cohort.csv'
# 'GvM_cohort_undersampled.csv'
# 'MEN_cohort.csv'
# 'MEN_cohort_undersampled.csv'

# 1) GBM vs. Metastasis without undersampling
df_GvM = pd.read_csv('GvM_cohort.csv')
X_GvM = df_GvM.iloc[:, 1:]
y_GvM = df_GvM['Label']

print('1) GBM vs. Metastasis dataset: \n \
    No. of samples and No. of features: {0} \n \
    Proportions of GBM and Metastasis: {1} and {2} \n \
    '.format(X_GvM.shape, round((1 - np.mean(y_GvM)), 2), 
             round(np.mean(y_GvM), 2)))

# 2) GBM vs. Metastasis with undersampling
df_GvM_under = pd.read_csv('GvM_cohort_undersampled.csv')
X_GvM_under = df_GvM_under.iloc[:, 1:]
y_GvM_under = df_GvM_under['Label']

print('2) Undersampled GBM vs. Metastasis dataset: \n \
    No. of samples and No. of features: {0} \n \
    Proportions of GBM and Metastasis: {1} and {2} \n \
    '.format(X_GvM_under.shape, round((1 - np.mean(y_GvM_under)), 2), 
             round(np.mean(y_GvM_under), 2)))

# 3) Low- vs. High-grade meningioma without undersampling
df_MEN = pd.read_csv('MEN_cohort.csv')
X_MEN = df_MEN.iloc[:, 1:]
y_MEN = df_MEN['Label']

print('3) Low- vs. High-grade meningioma dataset: \n \
    No. of samples and No. of features: {0} \n \
    Proportions of Low-grade and High-grade: {1} and {2} \n \
    '.format(X_MEN.shape, round((1 - np.mean(y_MEN)), 2), 
             round(np.mean(y_MEN), 2)))

# 4) Low- vs. High-grade meningioma with undersampling
df_MEN_under = pd.read_csv('MEN_cohort_undersampled.csv')
X_MEN_under = df_MEN_under.iloc[:, 1:]
y_MEN_under = df_MEN_under['Label']

print('4) Undersampled Low- vs. High-grade meningioma dataset: \n \
    No. of samples and No. of features: {0} \n \
    Proportions of Low-grade and High-grade: {1} and {2} \n \
    '.format(X_MEN_under.shape, round((1 - np.mean(y_MEN_under)), 2), 
             round(np.mean(y_MEN_under), 2)))


1) GBM vs. Metastasis dataset: 
     No. of samples and No. of features: (167, 558) 
     Proportions of GBM and Metastasis: 0.65 and 0.35 
     
2) Undersampled GBM vs. Metastasis dataset: 
     No. of samples and No. of features: (83, 558) 
     Proportions of GBM and Metastasis: 0.65 and 0.35 
     
3) Low- vs. High-grade meningioma dataset: 
     No. of samples and No. of features: (258, 186) 
     Proportions of Low-grade and High-grade: 0.63 and 0.37 
     
4) Low- vs. High-grade meningioma dataset: 
     No. of samples and No. of features: (129, 186) 
     Proportions of Low-grade and High-grade: 0.63 and 0.37 
     


## 1) Model stability by No. of input variables, task difficulty, and No. of samples

In [4]:
def get_auc_by_nfeat(file_data, cv, combs):

  df = pd.read_csv(file_data)
  X = df.iloc[:, 1:]
  y = df.loc[:, 'Label']
    
  result_df = pd.DataFrame()
  for (rs, n_feat) in tqdm(combs):
    # splitting
    X_train, X_test, y_train, y_test = \
      train_test_split(X, y, test_size=0.3, stratify=y, shuffle=True, random_state=rs)

    # standardization
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train, y_train)
    X_test = scaler.transform(X_test)

    # feature selection
    selector = SelectKBest(f_classif, k=n_feat)
    X_train = selector.fit_transform(X_train, y_train)
    X_test = selector.transform(X_test)

    # cross-validated AUC in the training set
    clf =  LogisticRegression(penalty='l1', solver='liblinear', max_iter=10000, 
                              dual=False, random_state=0)
    CV_score = cross_val_score(clf, X_train, y_train, scoring='roc_auc', cv=cv)
    CV_mean = np.mean(CV_score)
    CV_sd = np.std(CV_score)

    # model fitting and testing in the training and test sets, respectively
    clf.fit(X_train, y_train)
    y_pred = clf.predict_proba(X_test)
    test_auc = roc_auc_score(y_test, y_pred[:, 1])

    row = pd.DataFrame({'Random_state': rs,
                        'Num_features': n_feat,
                        'CV_AUC': round(CV_mean, 3), 
                        'CV_AUC_SD': round(CV_sd, 3),
                        'Test_AUC': round(test_auc, 3)}, index = [rs])
    result_df = pd.concat([result_df, row], axis=0)
  
  return result_df

In [5]:
combs = [(rs, n_feat) 
         for rs in range(1000) 
         for n_feat in range(1, 151)]

In [ ]:
get_auc_by_nfeat('GvM_cohort.csv', 
                 cv=5, combs=combs).to_csv('GvM_full_by_nfeat.csv')

In [ ]:
get_auc_by_nfeat('GvM_cohort_undersampled.csv', 
                 cv=5, combs=combs).to_csv('GvM_under_by_nfeat.csv')

In [ ]:
get_auc_by_nfeat('MEN_cohort.csv', 
                 cv=5, combs=combs).to_csv('MEN_full_by_nfeat.csv')

In [ ]:
get_auc_by_nfeat('MEN_cohort_undersampled.csv', 
                 cv=5, combs=combs).to_csv('MEN_under_by_nfeat.csv')

## 2) Model stability with hyperparameter tuning

In [8]:
def get_auc_gridsearch(file_data, rep, cv, params):

  df = pd.read_csv(file_data)
  X = df.iloc[:, 1:]
  y = df.loc[:, 'Label']

  result_df = pd.DataFrame()
  for rs in tqdm(range(rep)):
    X_train, X_test, y_train, y_test = \
          train_test_split(X, y, test_size=0.3, stratify=y, 
                          shuffle=True, random_state=rs)

    pipe = Pipeline([('scaler', StandardScaler()), 
                    ('fs', SelectKBest(f_classif)),
                    ('clf', LogisticRegression(penalty='l1', solver='liblinear',
                            max_iter=10000, dual=False, random_state=0))])

    search = GridSearchCV(pipe, param_grid=params, scoring='roc_auc', cv=cv)
    search.fit(X_train, y_train)

    best_nfeat = search.best_params_['fs__k']
    best_C = search.best_params_['clf__C']
    
    auc_list = []
    for j in range(cv): 
        key = 'split'+ str(j) + '_test_score'
        auc = search.cv_results_[key][search.best_index_]
        auc_list.append(auc)

    CV_AUC = np.mean(auc_list)
    CV_AUC_SD = np.std(auc_list)

    y_pred = search.predict_proba(X_test)
    Test_AUC = roc_auc_score(y_test, y_pred[:, 1]) 

    auc, auc_cov = delong_roc_variance(y_test, y_pred[:, 1])
    auc_std = np.sqrt(auc_cov)
    lower_upper_q = np.abs(np.array([0, 1]) - (1 - 0.95) / 2)
    ci = stats.norm.ppf(lower_upper_q, loc=auc, scale=auc_std)
    ci[ci > 1] = 1

    row = pd.DataFrame({'Random_state': rs,
                        'Optimal_Nfeat': best_nfeat,
                        'Optimal_C': best_C,
                        'CV_AUC': CV_AUC, 'CV_AUC_SD': CV_AUC_SD,
                        'Test_AUC': Test_AUC,
                        'CV_AUC_lowCI': ci[0], 
                        'CV_AUC_highCI': ci[1]}, index = [0])

    result_df = pd.concat([result_df, row], axis=0)

  return result_df

In [9]:
# define a hyperparameter grid for the ML model
C_list = list(np.arange(0.01, 0.1, 0.02)) + \
         list(np.arange(0.1, 1, 0.2)) + \
         list(np.arange(1, 2, 0.5)) + \
         list(np.arange(2, 10, 2))
params = {'fs__k': range(20, 55, 5), 'clf__C': C_list}

In [ ]:
get_auc_gridsearch('GvM_cohort.csv', 
 rep=1000, cv=5, params=params).to_csv('GvM_full_search.csv')

In [ ]:
get_auc_gridsearch('GvM_cohort_undersampled.csv', 
 rep=1000, cv=5, params=params).to_csv('GvM_under_search.csv')

In [ ]:
get_auc_gridsearch('MEN_cohort.csv', 
 rep=1000, cv=5, params=params).to_csv('MEN_full_search.csv')

In [ ]:
get_auc_gridsearch('MEN_cohort_undersampled.csv', 
 rep=1000, cv=5, params=params).to_csv('MEN_under_search.csv')